# Surface Warming Sensitivity and Synergy

By: Ty Janoski

In this script, we will attempt to calculate the surface warming sensitivity and synergy terms, as outlined in Laine et al. 2016.

In [1]:
# import statements

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import xesmf as xe

%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [2]:
# let's grab the CO2 strat-adjusted RF output from CAM4

# read in data file
fin = xr.open_dataset('/dx02/janoski/cesm-LE/strat_adj_rf/CAM4_CAMRT_strat_adj_RF.nc',use_cftime=True)

# we need lat and lon from any CESM-LE file
dummy = xr.open_dataset('/dx02/janoski/cesm-LE/output/b40.1850.cam5-lens.4xCO2.01.01.h1_PS.nc')


# make dataarray to regrid too
ds_out = xr.Dataset({'lat': (['lat'], dummy.lat),
                     'lon': (['lon'], dummy.lon),
                    }
                   )

# make regridder
regridder = xe.Regridder(fin, ds_out, 'bilinear',periodic=True)

# get difference b/w clear-sky and all-sky fluxes
# note: longwave has to be multiplied by -1 for directional consistency
RFLW_SFC = -1 *  regridder(fin.FLNS)
RFSW_SFC = regridder(fin.FSNS)

/home/janoski/miniconda3/envs/esmpy_env/lib/python3.7/site-packages/xesmf/frontend.py:478: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs,


In [5]:
# If my understanding is correct, we are going to calculate the sum of all the other radiative perturbations...

Ta = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.01.h1_Ta_rad_perturbs_sfc.nc',
                    use_cftime=True)
alb = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.01.h1_alb_rad_perturbs.nc',
                    use_cftime=True)
q = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.01.h1_Q_rad_perturbs.nc',
                    use_cftime=True)
cldlw = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.01.h1_cloud_LW_perturb_CAM4_sfc.nc',
                    use_cftime=True)
cldsw = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.01.h1_cloud_SW_perturb_CAM4_sfc.nc',
                    use_cftime=True)
ohu = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.01.h1_Fsfc_perturbs.nc',
                    use_cftime=True)
sfc_flx = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.01.h1_LH_and_SH_flux.nc',
                    use_cftime=True)
Ts_kern = np.load('/dx05/janoski/d10/Arctic_Research/cesm-LE/daily_kernels/ts.kernel.FLNS.npy')

In [7]:
gw = xr.open_dataarray('/dx02/janoski/cesm-LE/output/cam5_gauss_weights.nc')
# create function for taking spatial averages, while weighting for latitude
def gw_mean(ds_in, gw, lat_bound_s = -91, lat_bound_n = 91):
    """
    Use xarray/numpy to calculate spatial average while weighting for latitude.
    
    Keyword arguments:
    ds_in -- Dataset or DataArray to take the average of, ALREADY ZONALLY AVERAGED
    gw -- Array of guassian weights. Should only have latitude dimension.
    lat_bound_s -- float, Southern boundary of area to average
    lat_bound_n -- float, Northern boundary of area to average
    """
    return (ds_in.sel(lat=slice(lat_bound_s,lat_bound_n)) * gw.sel(lat=slice(lat_bound_s,lat_bound_n))/gw.sel(
        lat=slice(lat_bound_s,lat_bound_n)).sum(dim='lat')).sum(dim='lat')

In [22]:
ΣR = Ta.FLNS + alb.FSNS.fillna(0) + q.FLNS + q.FSNS + cldlw.cloud_LW + cldsw.cloud_SW + ohu.dFsfc + sfc_flx.total

In [26]:
ohu.dFsfc.isel(time=-1).plot()

<Figure size 432x288 with 2 Axes>